# GMC Demand Forecasting Tool

This tool analyzes your historical GMC reports to:
1. **Learn elasticities** - Reverse-engineer the demand formula from your data
2. **Predict demand** - Forecast orders for next period based on your decisions
3. **Optimize prices** - Find optimal prices to match production targets

---

## Quick Start

1. Put your historical Excel reports in `data/historical/` folder
2. Run all cells below
3. View results in `output/` folder

In [ ]:
# Import libraries
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our custom modules
sys.path.append('modules')
from data_loader import GMCReportLoader
from feature_engineering import GMCFeatureEngineer
from elasticity_estimator import ElasticityEstimator
from demand_predictor import DemandPredictor

# Setup
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ All modules loaded successfully!")

---

## Step 1: Load Historical Data

Load all your historical period reports from Excel files.

In [ ]:
# Configure paths
DATA_DIR = Path('data/historical')
OUTPUT_DIR = Path('output')
OUTPUT_DIR.mkdir(exist_ok=True)

# Load data
print("Loading historical reports...\n")
loader = GMCReportLoader()
df_raw = loader.load_multiple_reports(DATA_DIR)

print(f"\n✅ Loaded {len(df_raw)} periods of data")
print(f"   Periods: {df_raw['period'].tolist()}")
print(f"   Scenario: {df_raw['scenario'].iloc[0] if 'scenario' in df_raw.columns else 'Unknown'}")

# Display summary
df_raw[['period', 'year', 'quarter', 'group', 'company_id']].head(10)

---

## Step 2: Engineer Features

Calculate derived metrics (cumulative effects, quality scores, market saturation, etc.)

In [ ]:
print("Engineering features...\n")
engineer = GMCFeatureEngineer()
df = engineer.engineer_features(df_raw)

print(f"✅ Added derived features")
print(f"   Total columns: {len(df.columns)}")

# Show some key engineered features
key_features = [
    'period',
    'mgmt_budget_cumulative',
    'training_cumulative',
    'img_ad_eaec_cumulative',
    'quality_p1',
    'saturation_p1_eaec'
]

available_features = [f for f in key_features if f in df.columns]
if available_features:
    df[available_features].head()

---

## Step 3: Estimate Elasticities

Use regression to learn demand formula coefficients from your historical data.

In [ ]:
print("Estimating elasticities...\n")
print("="*60)

estimator = ElasticityEstimator()
elasticity_results = estimator.estimate_all(df)

print("\n" + "="*60)
print("\n✅ Elasticity estimation complete!")

# Save results
output_file = OUTPUT_DIR / 'elasticities.xlsx'
estimator.save_results(elasticity_results, str(output_file))

In [ ]:
# Visualize elasticities
print("\n📊 Elasticity Summary:\n")

# Create summary table
summary_data = []
for key, elast_dict in elasticity_results['elasticities'].items():
    row = {
        'Product-Market': key.upper(),
        'Base Demand': f"{elasticity_results['base_demands'][key]:.0f}",
        'R²': f"{elasticity_results['model_quality'][key]:.3f}",
        'Price Effect': f"{elast_dict.get('log_price_ratio', 0):.3f}",
        'Quality Effect': f"{elast_dict.get('log_quality', 0):.3f}",
        'Image Ad': f"{elast_dict.get('img_ad_cumulative', 0):.5f}",
        'Direct Ad': f"{elast_dict.get('dir_ad_cumulative', 0):.5f}",
    }
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print("\n💡 Interpretation:")
print("   - Negative price effect = Lower price → Higher demand (expected)")
print("   - Positive quality effect = Better quality → Higher demand")
print("   - R² shows model fit (0.7+ is good, 0.9+ is excellent)")

---

## Step 4: Predict Demand for Next Period

Input your planned decisions and get demand forecasts.

In [ ]:
# Get current period (last historical period)
current_period = df.iloc[-1]
next_period_number = int(current_period['period']) + 1

print(f"Predicting demand for Period {next_period_number}")
print(f"Based on data from Period {int(current_period['period'])}\n")

In [ ]:
# Define your planned decisions for next period
# Modify these values based on your strategy!

next_period_decisions = pd.Series({
    # Prices (3 products x 3 markets)
    'price_p1_eaec': 1100,
    'price_p1_eu': 1150,
    'price_p1_internet': 1080,
    'price_p2_eaec': 1250,
    'price_p2_eu': 1300,
    'price_p2_internet': 1200,
    'price_p3_eaec': 1450,
    'price_p3_eu': 1500,
    'price_p3_internet': 1400,
    
    # Image advertising by market
    'img_ad_eaec': 75,
    'img_ad_eu': 75,
    'img_ad_internet': 80,
    
    # Direct advertising (sample for product 1)
    'dir_ad_p1_eaec': 30,
    'dir_ad_p1_eu': 30,
    'dir_ad_p1_internet': 35,
    'dir_ad_p2_eaec': 25,
    'dir_ad_p2_eu': 25,
    'dir_ad_p2_internet': 30,
    'dir_ad_p3_eaec': 20,
    'dir_ad_p3_eu': 20,
    'dir_ad_p3_internet': 25,
    
    # Management & Training
    'mgmt_budget': 180,
    'training_days': 35,
    
    # R&D spending
    'rd_spend_p1': 60,
    'rd_spend_p2': 60,
    'rd_spend_p3': 60,
    
    # Assembly time (% of standard 300 min)
    'assembly_time_p1': 330,  # 110%
    'assembly_time_p2': 330,
    'assembly_time_p3': 330,
    
    # Premium materials %
    'premium_p1': 20,
    'premium_p2': 20,
    'premium_p3': 20,
    
    # A&D
    'agents_eaec_next': 6,
    'agents_eaec_commission': 9,
    'agents_eu_next': 6,
    'agents_eu_commission': 9,
    'internet_agent_reward': 15,
    'internet_agent_commission': 6,
    
    # Internet
    'internet_ports': 18,
    'website_dev': 40,
    
    # Quality (use current values or expected values)
    'quality_p1': current_period.get('quality_p1', 96),
    'quality_p2': current_period.get('quality_p2', 96),
    'quality_p3': current_period.get('quality_p3', 96),
})

print("✅ Decisions configured")
print(f"\nKey decisions:")
print(f"  Prices P1: {next_period_decisions['price_p1_eaec']}, {next_period_decisions['price_p1_eu']}, {next_period_decisions['price_p1_internet']}")
print(f"  Mgmt Budget: {next_period_decisions['mgmt_budget']}")
print(f"  Training: {next_period_decisions['training_days']} days")

In [ ]:
# Make predictions
predictor = DemandPredictor(elasticity_results)
predictions = predictor.predict_demand(
    current_period,
    next_period_decisions,
    next_period_number
)

# Display results
print(f"\n📈 DEMAND FORECAST FOR PERIOD {next_period_number}\n")
print("="*80)

display_cols = ['product', 'market', 'predicted_demand', 'seasonality_factor', 
                'price_effect', 'quality_effect']
print(predictions[display_cols].to_string(index=False))

# Save predictions
output_file = OUTPUT_DIR / 'demand_forecast.xlsx'
predictions.to_excel(output_file, index=False)
print(f"\n✅ Forecast saved to {output_file}")

---

## Step 5: Scenario Analysis

Test different decision combinations to see impact on demand.

In [ ]:
# Define scenarios to test
scenarios = {
    'price_minus_5pct': {
        'price_p1_eaec': next_period_decisions['price_p1_eaec'] * 0.95,
        'price_p1_eu': next_period_decisions['price_p1_eu'] * 0.95,
        'price_p1_internet': next_period_decisions['price_p1_internet'] * 0.95,
    },
    'price_plus_5pct': {
        'price_p1_eaec': next_period_decisions['price_p1_eaec'] * 1.05,
        'price_p1_eu': next_period_decisions['price_p1_eu'] * 1.05,
        'price_p1_internet': next_period_decisions['price_p1_internet'] * 1.05,
    },
    'high_advertising': {
        'img_ad_eaec': 99,
        'img_ad_eu': 99,
        'img_ad_internet': 99,
        'dir_ad_p1_eaec': 50,
        'dir_ad_p1_eu': 50,
        'dir_ad_p1_internet': 50,
    },
    'low_advertising': {
        'img_ad_eaec': 40,
        'img_ad_eu': 40,
        'img_ad_internet': 40,
        'dir_ad_p1_eaec': 15,
        'dir_ad_p1_eu': 15,
        'dir_ad_p1_internet': 15,
    },
}

# Run scenario analysis
scenario_results = predictor.predict_with_scenarios(
    current_period,
    next_period_decisions,
    scenarios,
    next_period_number
)

# Show results for Product 1 only
print("\n📊 SCENARIO ANALYSIS - Product 1\n")
p1_scenarios = scenario_results[scenario_results['product'] == 'p1']

pivot = p1_scenarios.pivot_table(
    index='market',
    columns='scenario',
    values='predicted_demand',
    aggfunc='first'
)

print(pivot.to_string())

# Save scenario analysis
output_file = OUTPUT_DIR / 'scenario_analysis.xlsx'
scenario_results.to_excel(output_file, index=False)
print(f"\n✅ Scenario analysis saved to {output_file}")

---

## Step 6: Visualize Results

In [ ]:
# Plot predicted demand by product and market
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, market in enumerate(['eaec', 'eu', 'internet']):
    market_data = predictions[predictions['market'] == market]
    
    axes[idx].bar(market_data['product'], market_data['predicted_demand'])
    axes[idx].set_title(f'{market.upper()} Market')
    axes[idx].set_xlabel('Product')
    axes[idx].set_ylabel('Predicted Demand')
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'demand_forecast_chart.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Chart saved to output/demand_forecast_chart.png")

---

## Summary

All outputs saved to `output/` folder:
- `elasticities.xlsx` - Learned coefficients
- `demand_forecast.xlsx` - Demand predictions
- `scenario_analysis.xlsx` - Sensitivity analysis
- `demand_forecast_chart.png` - Visualization

---

## Next Steps

1. **Refine decisions**: Adjust values in Step 4 based on predictions
2. **Test scenarios**: Add more scenarios in Step 5
3. **Price optimization**: Use predicted elasticities to find optimal prices
4. **Update after each period**: Add new period data and re-run